In [1]:
# Colab settings
# !wget --header="Host: archive.ics.uci.edu" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7" --header="Referer: https://archive.ics.uci.edu/ml/machine-learning-databases/00363/" "https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip" -O "Dataset.zip" -c
# !unzip Dataset.zip
# !git clone https://github.com/janchk/ML_HOMEWORK1
# from ML_HOMEWORK_1.models import LinearRegressionWithGd
# from ML_HOMEWORK_1.common import *

In [2]:
#import part 
import matplotlib.pyplot as plt
from IPython import display
from models import LinearRegressionWithGd
from common import *
import pandas as pd
path = "Dataset/Training/"

In [3]:
# configuration
num_folds = 5
epochs = 1000
learning_rate = 0.1

lrg = LinearRegressionWithGd()
lrg.learning_rate = learning_rate

df_1, df_2 = folding(path)
np.random.shuffle(df_1)

### Training

In [4]:
features = []
bias = []
stats = []
losses = [] #loss per fold
for i in range(num_folds):
    train, test = cv_loo(df_1, num_folds, i)

    X_train = normalized(train.T[0:53]).T
    X_test = normalized(test.T[0:53]).T

    Y_train = train.T[53].T
    Y_train = np.expand_dims(Y_train, axis=-1)

    Y_test = test.T[53].T
    Y_test = np.expand_dims(Y_test, axis=-1)

    loss = lrg.train(epochs, X_train, Y_train)
    stat = lrg.validate(X_test, Y_test)
    losses.append(loss)
    stats.append(stat)
    features.append(lrg.betas)
    bias.append(*lrg.bias)
    

mean_stats = np.sum(stats, axis=0) / num_folds
print("mse_val is {}, r2_val is {}, rmse is {}, r2/rmse {}".format(
    mean_stats[0], mean_stats[1], mean_stats[2], mean_stats[3]))


TypeError: 'numpy.float64' object does not support item assignment

### Features

In [ ]:
columns = ["bias"] + [f"f{f+1}" for f in range(len(features[0]))]
index = [f"Fold {i+1}" for i, _ in enumerate(features)] + ["mean"] + ["std"]

bias_np = np.expand_dims(np.squeeze(bias), 1)
features_np = np.squeeze(features, axis=2)
features_np = np.hstack((bias, features_np))
features_mean = np.mean(features_np, axis=0)
features_std = np.std(features_np, axis=0)
df = pd.DataFrame(np.vstack((features_np, features_mean, features_std )),
                  columns=columns,
                  index=index)

display.display(df)    

### Stats

In [ ]:
stats_np = np.array(stats)
losses_np = np.expand_dims(np.array(losses), 1)
columns = ["mse_train"] + ["mse_val"] + ["r2_val"] + ["rmse_val"] + ["r2/rmse_val"]
index = [f"Fold {i+1}" for i, _ in enumerate(stats_np)] # + ["mean"] + ["std"]
all_stats = np.hstack((losses_np, stats_np))
df = pd.DataFrame(all_stats,
                  columns=columns,
                  index=index)

display.display(df) 

MD
